# Models

In [ ]:
import numpy as np
import sympy as sp

# parameters
class PAR:
    l1 = 1.0 # length of pendulum 1
    l2 = 1.0 # length of pendulum 2
    g = 9.81 # gravity
    μ1 = 0.03 # damping coefficient
    μ2 = 0.03 # damping coefficient
    m1 = .1 # mass of pendulum 1
    m2 = .1 # mass of pendulum 2
    mc = .1 # mass of cart

l1, l2, g, μ1, μ2, m1, m2, mc = sp.symbols('l1 l2 g μ1 μ2 m1 m2 mc', real=True, positive=True)

In [ ]:
#save the file with the model equations
MODELS_EQ_PATH = 'models_eq.py'
with open(MODELS_EQ_PATH, 'w') as file:
    file.write('# This file has been automatically generated by models.ipynb\n')
    file.write(f'from utils import *\n\n')
    #save parameters
    file.write(f'# parameters\n')
    file.write(f'class PAR:\n')
    for k, v in vars(PAR).items():
        if not k.startswith('__'): file.write(f'    {k} = {v}\n')

In [ ]:
# utils
def sp2np(f:sp.Function):
    '''sketchy way to convert sympy expression to numpy'''
    f = f.simplify()
    f = str(f)
    subs = [('u(t)', 'u')]
    for i in range(1, 7):
        subs.append((f'Derivative(s{i}(t), (t, 3))', f'ddds{i}'))
        subs.append((f'Derivative(s{i}(t), (t, 2))', f'dds{i}'))
        subs.append((f'Derivative(s{i}(t), (t, 1))', f'ds{i}'))
        subs.append((f'Derivative(s{i}(t), t)', f'ds{i}')) 
        subs.append((f's{i}(t)', f's{i}'))
    for s in subs: f = f.replace(*s) # replace all the terms
    return f

def mat2np(A:sp.Matrix):
    n, m = A.shape
    Ap = np.zeros((n, m), dtype=object) # will be an array of strings
    for i in range(n):
        for j in range(m):
            Ap[i, j] = sp2np(A[i, j])
    Ap = str(Ap).replace("' '", ", ") # remove the quotes
    Ap = Ap.replace("'", "") # remove the quotes
    Ap = Ap.replace("\n", ",") # remove the spaces
    return Ap

def sub_params(f):
    return f.subs([(l1, PAR.l1), (l2, PAR.l2), (g, PAR.g), (μ1, PAR.μ1), (μ2, PAR.μ2), (m1, PAR.m1), (m2, PAR.m2), (mc, PAR.mc)])
                   
# Taylor approximation at s0 of the function 'f'
def taylor(f:sp.Function,s:sp.Symbol,s0:float,n:int):
    def factorial(n): return 1 if n == 0 else n*factorial(n-1)
    return sum(f.diff(s,i).subs(s,s0)/(factorial(i))*(s-s0)**i for i in range(n+1))

def linear_system_lagrangian(fs:list[sp.Function], states:list[sp.Symbol], u:sp.Symbol, s_eq:list[float]):
    # linearize the system around the equilibrium point s_eq
    # assumes that the fs equations are in the form d2_s = f(s, u), d2_s: second derivative of s
    # NOTE: states must be in the order with states first: states= [s1, s2, ..., sn, ds1, ds2, ..., dsn]
    nf, ns = len(fs), len(states)
    assert len(s_eq) == ns, f"number of states must be equal to the number of equilibrium points, got {ns} and {len(s_eq)}"
    assert 2*nf == ns, f"number of states must be double the number of equations, got {ns} and {nf}"
    # system matrix
    A = sp.zeros(2*nf, 2*nf) 
    # remove dependency on the derivatives 
    oldds, newds = states[nf:], [sp.Symbol(f'd{i+1}(t)', real=True) for i in range(nf)] # old derivatives, new derivatives
    for i in range(nf):  
        for j in range(nf): fs[i] = fs[i].subs(oldds[j], newds[j])
    states[nf:] = newds # replace the old derivatives with the new ones
    for i in range(nf): # calculate the coefficients
        i1, i2 = i, i+nf 
        f, x, dx, eqx, eqdx = fs[i], states[i1], states[i2], s_eq[i1], s_eq[i2]
        f = f.subs(u, 0) #subs input first
        fx = taylor(f, x, eqx, 1) # taylor expansion around eqx
        fdx = taylor(f, dx, eqdx, 1) # taylor expansion around eqdx
        for j in reversed(range(ns)): # replace all the states, derivatives first
            fx = fx.subs(states[j], 1 if j == i1 else 0)#; print(f'subs: j:{j}, s[j]: {states[j]}, i1: {i1}, i2: {i2}, fx: {fx}')
            fdx = fdx.subs(states[j], 1 if j == i2 else 0)#; print(f'subs: j:{j}, s[j]: {states[j]}, i1: {i1}, i2: {i2}, fdx: {fdx}')
        A[i, i+nf], A[i+nf, i], A[i+nf, i+nf] = 1, fx, fdx # set the values in the matrix
    # input matrix
    B = sp.zeros(2*nf, 1)
    for i in range(nf):
        f = fs[i]
        for s in states: f = f.subs(s, 0)
        f = f.subs(u, 1)
        B[i+nf, 0] = f # set the value in the matrix 
    # ouput matrix
    C = sp.zeros(nf, 2*nf) # output matrix returns the states (not the derivatives)
    for i in range(nf): C[i, i], C[i, i+nf] = 1, 0 # set the values in the matrix   
    return A, B, C

In [ ]:
# single pendulum
t = sp.symbols('t', real=True)
u = sp.symbols('u', cls=sp.Function, real=True)(t)

# single pendulum
s1 = sp.symbols('s1', cls=sp.Function, real=True)(t) 
ds1 = sp.diff(s1, t)
dds1 = sp.diff(ds1, t)

V = m1 * g * l1 * sp.cos(s1) # potential energy
T = 0.5 * m1 * (l1 * ds1)**2 # kinetic energy
L = T - V # lagrangian
print('\nLagrangian:')
print(sp2np(L))

L1 = L.diff(s1) - (L.diff(ds1)).diff(t) - μ1 * ds1 + u
print('\nEuler-Lagrange equation:')
print(sp2np(L1))

# solve
sol = sp.solve(L1, dds1, simplify=False)
dds1 = sol[0]
print('\nSolved:')
print(f'dds1 == {sp2np(dds1)}')

# linearize around s1=0
A, B, C = linear_system_lagrangian([dds1], [s1, ds1], u, [0, 0])
print('\nLinearized system:')
print(f'A:\n{mat2np(A)}')
print(f'B:\n{mat2np(B)}')
print(f'C:\n{mat2np(C)}')

# sobstitute parameters
dds1, A, B = sub_params(dds1), sub_params(A), sub_params(B)
print('\nSubstituted parameters:')
print(sp2np(dds1))
print(mat2np(A))
print(mat2np(B))

# save the model to file
with open(MODELS_EQ_PATH, 'a') as file:
    file.write(f'\n# single pendulum\n')
    file.write(f'class SinglePendulum():\n')
    file.write(f'    A = vec({mat2np(A)})\n')
    file.write(f'    B = vec({mat2np(B)})\n')
    file.write(f'    C = vec({mat2np(C)})\n')
    file.write(f'    @staticmethod\n')
    file.write(f'    def f(s1, ds1, u):\n')
    file.write(f'        return [{sp2np(dds1)}]\n')
    file.write(f'    LAB = [\'θ\', \'dθ\']\n')

In [ ]:
# double pendulum
s1 = sp.symbols('s1', cls=sp.Function, real=True)(t)
s2 = sp.symbols('s2', cls=sp.Function, real=True)(t)
ds1, ds2 = sp.diff(s1, t), sp.diff(s2, t)
dds1, dds2 = sp.diff(ds1, t), sp.diff(ds2, t)
x1, y1 = l1 * sp.sin(s1), l1 * sp.cos(s1)
x2, y2 = x1 + l2 * sp.sin(s2), y1 + l2 * sp.cos(s2)
dx1, dy1 = sp.diff(x1, t), sp.diff(y1, t)
dx2, dy2 = sp.diff(x2, t), sp.diff(y2, t)

V = m1 * g * y1 + m2 * g * y2 # potential energy
T = 0.5 * m1 * (dx1**2 + dy1**2) + 0.5 * m2 * (dx2**2 + dy2**2) # kinetic energy
L = T - V # lagrangian
print('\nLagrangian:')
print(sp2np(L))
L1 = L.diff(s1) - (L.diff(ds1)).diff(t) - μ1 * ds1 + u
L2 = L.diff(s2) - (L.diff(ds2)).diff(t) - μ2 * ds2

print('\nEuler-Lagrange equation:')
print(sp2np(L1))
print(sp2np(L2))
#  equations are almost too long to solve without substituting the actual parameters -> substitute first
L1, L2 = sub_params(L1), sub_params(L2)

# solve
sol = sp.solve([L1, L2], [dds1, dds2], simplify=False)
dds1, dds2 = sol[dds1], sol[dds2]
print('\nSolved:')
print(f'dds1 == {dds1}')
print(f'dds2 == {dds2}')
print(f'dds1 == {sp2np(dds1)}')
print(f'dds2 == {sp2np(dds2)}')

# linearize around s1=s2=0
A, B, C = linear_system_lagrangian([dds1, dds2], [s1, s2, ds1, ds1], u, [0, 0, 0, 0])
print('\nLinearized system:')
print(f'A:\n{mat2np(A)}')
print(f'B:\n{mat2np(B)}')
print(f'C:\n{mat2np(C)}')


# sobstitute parameters
dds1, dds2, A, B = sub_params(dds1), sub_params(dds2), sub_params(A), sub_params(B)
print('\nSubstituted parameters:')
print(sp2np(dds1))
print(sp2np(dds2))

# save the model to file
with open(MODELS_EQ_PATH, 'a') as file:
    file.write(f'\n# double pendulum\n')
    file.write(f'class DoublePendulum():\n')
    file.write(f'    A = vec({mat2np(A)})\n')
    file.write(f'    B = vec({mat2np(B)})\n')
    file.write(f'    C = vec({mat2np(C)})\n')
    file.write(f'    @staticmethod\n')
    file.write(f'    def f(s1, s2, ds1, ds2, u):\n')
    file.write(f'        return [{sp2np(dds1)}, {sp2np(dds2)}]\n')
    file.write(f'    LAB = [\'θ1\', \'θ2\', \'dθ1\', \'dθ2\']\n')

In [ ]:
# cart single pendulum
s1 = sp.symbols('s1', cls=sp.Function, real=True)(t) # angle
s2 = sp.symbols('s2', cls=sp.Function, real=True)(t) # position
ds1, ds2 = sp.diff(s1, t), sp.diff(s2, t)
dds1, dds2 = sp.diff(ds1, t), sp.diff(ds2, t)
x1, y1 = l1 * sp.sin(s1) + s2, l1 * sp.cos(s1)
dx1, dy1 = sp.diff(x1, t), sp.diff(y1, t)

V = m1 * g * y1 # potential energy
T = 0.5 * m1 * (dx1**2 + dy1**2) + 0.5 * mc * ds2**2 # kinetic energy
L = T - V # lagrangian
print('\nLagrangian:')
print(sp2np(L))

L1 = L.diff(s1) - (L.diff(ds1)).diff(t) - μ1 * ds1 
L2 = L.diff(s2) - (L.diff(ds2)).diff(t) + u/mc
print('\nEuler-Lagrange equation:')
print(sp2np(L1))
print(sp2np(L2))

# solve
sol = sp.solve([L1, L2], [dds1, dds2], simplify=False)
dds1, dds2 = sol[dds1], sol[dds2]
print('\nSolved:')
print(f'dds1 == {dds1}')
print(f'dds2 == {dds2}')
print(f'dds1 == {sp2np(dds1)}')
print(f'dds2 == {sp2np(dds2)}')

# linearize around s1=0
A, B, C = linear_system_lagrangian([dds1, dds2], [s1, s2, ds1, ds2], u, [0, 0, 0, 0])
print('\nLinearized system:')
print(f'A:\n{mat2np(A)}')
print(f'B:\n{mat2np(B)}')
print(f'C:\n{mat2np(C)}')

# sobstitute parameters
dds1, dds2, A, B = sub_params(dds1), sub_params(dds2), sub_params(A), sub_params(B)
print('\nSubstituted parameters:')
print(sp2np(dds1))
print(sp2np(dds2))
print(mat2np(A))
print(mat2np(B))

# save the model to file
with open(MODELS_EQ_PATH, 'a') as file:
    file.write(f'\n# cart single pendulum\n')
    file.write(f'class CartSinglePendulum():\n')
    file.write(f'    A = vec({mat2np(A)})\n')
    file.write(f'    B = vec({mat2np(B)})\n')
    file.write(f'    C = vec({mat2np(C)})\n')
    file.write(f'    @staticmethod\n')
    file.write(f'    def f(s1, s2, ds1, ds2, u):\n')
    file.write(f'        return [{sp2np(dds1)}, {sp2np(dds2)}]\n')
    file.write(f'    LAB = [\'θ\', \'x\', \'dθ\', \'dx\']\n')

In [ ]:
# cart double pendulum
s1 = sp.symbols('s1', cls=sp.Function, real=True)(t) # angle first joint
s2 = sp.symbols('s2', cls=sp.Function, real=True)(t) # angle second joint
s3 = sp.symbols('s3', cls=sp.Function, real=True)(t) # position of cart
ds1, ds2, ds3 = sp.diff(s1, t), sp.diff(s2, t), sp.diff(s3, t)
dds1, dds2, dds3 = sp.diff(ds1, t), sp.diff(ds2, t), sp.diff(ds3, t)

x1, y1 = l1 * sp.sin(s1) + s3, l1 * sp.cos(s1)
x2, y2 = x1 + l2 * sp.sin(s2), y1 + l2 * sp.cos(s2)
dx1, dy1 = sp.diff(x1, t), sp.diff(y1, t)
dx2, dy2 = sp.diff(x2, t), sp.diff(y2, t)

V = m1 * g * y1 + m2 * g * y2 # potential energy
T = 0.5 * m1 * (dx1**2 + dy1**2) + 0.5 * m2 * (dx2**2 + dy2**2) + 0.5 * mc * ds3**2 # kinetic energy
L = T - V # lagrangian

print('\nLagrangian:')
print(sp2np(L))

L1 = L.diff(s1) - (L.diff(ds1)).diff(t) - μ1 * ds1
L2 = L.diff(s2) - (L.diff(ds2)).diff(t) - μ2 * ds2
L3 = L.diff(s3) - (L.diff(ds3)).diff(t) + u/mc

print('\nEuler-Lagrange equation:')
print(sp2np(L1))
print(sp2np(L2))
print(sp2np(L3))

# equations are too long to solve without substituting the actual parameters -> substitute first
L1, L2, L3 = sub_params(L1), sub_params(L2), sub_params(L3)

# solve
sol = sp.solve([L1, L2, L3], [dds1, dds2, dds3], simplify=False)
dds1, dds2, dds3 = sol[dds1], sol[dds2], sol[dds3]
print('\nSolved:')
print(f'dds1 == {dds1}')
print(f'dds2 == {dds2}')
print(f'dds3 == {dds3}')
print(f'dds1 == {sp2np(dds1)}')
print(f'dds2 == {sp2np(dds2)}')
print(f'dds3 == {sp2np(dds3)}')

# linearize around s1=s2=0
A, B, C = linear_system_lagrangian([dds1, dds2, dds3], [s1, s2, s3, ds1, ds2, ds3], u, [0, 0, 0, 0, 0, 0])
print('\nLinearized system:')
print(f'A:\n{mat2np(A)}')
print(f'B:\n{mat2np(B)}')
print(f'C:\n{mat2np(C)}')
# sobstitute parameters
dds1, dds2, dds3, A, B = sub_params(dds1), sub_params(dds2), sub_params(dds3), sub_params(A), sub_params(B)
print('\nSubstituted parameters:')
print(sp2np(dds1))
print(sp2np(dds2))
print(sp2np(dds3))
print(mat2np(A))
print(mat2np(B))

# save the model to file
with open(MODELS_EQ_PATH, 'a') as file:
    file.write(f'\n# cart double pendulum\n')
    file.write(f'class CartDoublePendulum():\n')
    file.write(f'    A = vec({mat2np(A)})\n')
    file.write(f'    B = vec({mat2np(B)})\n')
    file.write(f'    C = vec({mat2np(C)})\n')
    file.write(f'    @staticmethod\n')
    file.write(f'    def f(s1, s2, s3, ds1, ds2, ds3, u):\n')
    file.write(f'        return [{sp2np(dds1)}, {sp2np(dds2)}, {sp2np(dds3)}]\n')
    file.write(f'    LAB = [\'θ1\', \'θ2\', \'x\', \'dθ1\', \'dθ2\', \'dx\']\n')

## Test the integration

In [ ]:
# test the integration
import numpy as np
import matplotlib.pyplot as plt
from models_eq import SinglePendulum as SP, DoublePendulum as DP, CartSinglePendulum as CSP, CartDoublePendulum as CDP
from utils import *
FPS = 60.0

# simulation parameters
dt = 0.0001 # time step
T  = .5 # simulation time
# T  = 5 # simulation time
nt = int(T / dt) # number of time steps

θ = 0.01 # initial angle

uλ = lambda i: 0.2*sin(3*π*T*i/nt) # control input
# uλ = lambda i: 0 # control input

cols = ['b', 'g', 'r', 'c', 'm', 'y'] # colors

# single pendulum
ss = np.zeros((nt, 2)) # states
lss = np.zeros((nt, 2)) # linear states
ss[0] = [θ, 0.0] # initial state
lss[0] = [θ, 0.0] # initial state
for i in range(1, nt): ss[i] = step(SP.f, ss[i-1], uλ(i), dt) # integrate
for i in range(1, nt): lss[i] = lstep(SP.A, SP.B, lss[i-1], vec([uλ(i)]), dt) # integrate linear system
lims_nl = (1.1*np.max(ss), 1.1*np.min(ss))
# plot
plt.figure(figsize=(15, 4))
for i in range(2):
    plt.plot(ss[:, i], color=cols[i], label=SP.LAB[i])
    plt.plot(lss[:, i], color=cols[i], linestyle='--', label=f'linear {SP.LAB[i]}')
plt.ylim(lims_nl)
plt.legend()
plt.title('Single Pendulum')
plt.show()

# double pendulum
ss = np.zeros((nt, 4)) # states
lss = np.zeros((nt, 4)) # linear states
ss[0] = lss[0] = [θ, -θ, 0.0, 0.0] # initial state
for i in range(1, nt): ss[i] = step(DP.f, ss[i-1], uλ(i), dt) # integrate
for i in range(1, nt): lss[i] = lstep(DP.A, DP.B, lss[i-1], vec([uλ(i)]), dt) # integrate linear system
lims_nl = (1.1*np.max(ss), 1.1*np.min(ss))
# plot
plt.figure(figsize=(15, 4))
for i in range(4):
    plt.plot(ss[:, i], color=cols[i % len(cols)], label=DP.LAB[i])
    plt.plot(lss[:, i], color=cols[i % len(cols)], linestyle='--', label=f'linear {DP.LAB[i]}')
plt.ylim(lims_nl)
plt.legend()
plt.title('Double Pendulum')
plt.show()

# cart single pendulum
ss = np.zeros((nt, 4)) # states
lss = np.zeros((nt, 4)) # linear states
ss[0] = lss[0] = [θ, 0.01, 0.0, 0.0] # initial state
for i in range(1, nt): ss[i] = step(CSP.f, ss[i-1], uλ(i), dt) # integrate
for i in range(1, nt): lss[i] = lstep(CSP.A, CSP.B, lss[i-1], vec([uλ(i)]), dt) # integrate linear system
lims_nl = (1.1*np.max(ss), 1.1*np.min(ss))
# plot
plt.figure(figsize=(15, 4))
for i in range(4):
    plt.plot(ss[:, i], color=cols[i % len(cols)], label=CSP.LAB[i])
    plt.plot(lss[:, i], color=cols[i % len(cols)], linestyle='--', label=f'linear {CSP.LAB[i]}')
plt.ylim(lims_nl)
plt.legend()
plt.title('Cart Single Pendulum')
plt.show()

# cart double pendulum
ss = np.zeros((nt, 6)) # states
lss = np.zeros((nt, 6)) # linear states
ss[0] = lss[0] = [θ, -θ, 0.01, 0.0, 0.0, 0.0] # initial state
for i in range(1, nt): ss[i] = step(CDP.f, ss[i-1], uλ(i), dt) # integrate
for i in range(1, nt): lss[i] = lstep(CDP.A, CDP.B, lss[i-1], vec([uλ(i)]), dt) # integrate linear system
lims_nl = (1.1*np.max(ss), 1.1*np.min(ss))
# plot
plt.figure(figsize=(15, 4))
for i in range(6):
    plt.plot(ss[:, i], color=cols[i % len(cols)], label=CDP.LAB[i])
    plt.plot(lss[:, i], color=cols[i % len(cols)], linestyle='--', label=f'linear {CDP.LAB[i]}')
plt.ylim(lims_nl)
plt.legend()
plt.title('Cart Double Pendulum')
plt.show()
